In [1]:
# TODO LIST FROM GANHACKS
# https://github.com/soumith/ganhacks
# ✓ Normalize the inputs
# ✓ A modified loss function
# ✓ Use a spherical Z
# ✓ BatchNorm
# Aviod sparse gradients
# ✓ Use soft and noisy labels
# ✓ DCGAN
# Use stability tricks from RL
# ✓ Use SGD for discriminator ADAM for generator
# ✓ Add noise to inputs
# Batch Discrimination (for diversity)
# ✓ Use dropouts in G in both train and test phase

In [2]:
import network as layer
import torch.nn as nn

nc = 3
nz = 512
nfeature = 736

In [3]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.utils.data
class ImageFeatureDataset(dset.ImageFolder):
    def __init__(self, root, embed_dir, transform=None):
        super().__init__(root=root, transform=transform)
        self.embed = torch.load(embed_dir)
#         self.landmark = torch.load(landmark_dir)
#         self.feature = torch.cat([self.embed, self.landmark], 1)
        self.feature = self.embed
    
    def __getitem__(self, index):
        img, target = super().__getitem__(index)
        feature = self.feature[index]
        return (img, feature)

In [4]:
lr     = 0.0002
beta1  = 0.0   
beta2  = 0.99     
imageSize = 64
batchSize = 64

outf = "./celeba_result/"
des_dir = "./celeba/"
embed_dir = "./celeba_embed/image_features.pth"

dataset = ImageFeatureDataset(root=des_dir,
                            embed_dir=embed_dir,
                            transform=transforms.Compose([
                                transforms.CenterCrop(224),
                                transforms.Resize(imageSize),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                            ]))

dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size= batchSize,
                                         shuffle=True)

In [5]:
from custom_layers import *

class Generator(nn.Module):
    def make_dense(self, k_in, k_growth, n, options):
        layers = []
        for i in range(n):
            layers.append(Dense(layer.conv(k_in, k_growth, 3, 1, 1, **options)))
            k_in += k_growth
        return nn.Sequential(*layers)
    
    def __init__(self):
        super(Generator, self).__init__()
        options = {'leaky':True, 'bn':True, 'wn':False, 'pixel':True, 'gdrop':True}
        layers = []
        
        layers.append(layer.deconv(nz + nfeature, 512, 4, 1, 0, **options))
        # 4 x 4
#         layers.append(layer.deconv(512, 512, 4, 2, 1, **options))
        # 8 x 8
        layers.append(layer.deconv(512, 256, 4, 2, 1, **options))
        # 16 x 16
        layers.append(layer.deconv(256, 128, 4, 2, 1, **options))
        # 32 x 32
        layers.append(layer.deconv(128, 64, 4, 2, 1, **options))
        # 64 x 64
        layers.append(layer.deconv(64, nc, 4, 2, 1, gdrop=options['gdrop'], only=True))
        # 128 x 128
        layers.append(nn.Tanh())
        
        self.network = nn.Sequential(*layers)
        
    def forward(self, x, feature):
        feature = feature.view(-1, nfeature, 1, 1)
        x = self.network(torch.cat([x, feature], 1))
        return x
    
netG = Generator()

In [6]:
class Discriminator(nn.Module):
    def make_dense(self, k_in, k_growth, n, options):
        layers = []
        for i in range(n):
            layers.append(Dense(layer.conv(k_in, k_growth, 3, 1, 1, **options)))
            k_in += k_growth
        return nn.Sequential(*layers)
    
    def __init__(self):
        super(Discriminator, self).__init__()
        
        options = {'leaky':True, 'bn':False, 'wn':False, 'pixel':False, 'gdrop':False}
        
        layers = []
        
        layers.append(layer.conv(nc, 64, 4, 2, 1, **options))
        # 64 x 64
        layers.append(layer.conv(64, 128, 4, 2, 1, **options))
        # 32 x 32
        layers.append(layer.conv(128, 256, 4, 2, 1, **options))
        # 16 x 16
        layers.append(layer.conv(256, 512, 4, 2, 1, **options))
        # 8 x 8
#         layers.append(layer.conv(512, 512, 4, 2, 1, **options))
        # 4 x 4
        layers.append(layer.conv(512, 512, 4, 1, 0, **options))
        # 1 x 1
#         layers.append(nn.Sigmoid())
        
        self.network = nn.Sequential(*layers)
        self.linear = layer.linear(512 + nfeature, 1, sig=False, wn=options['wn'])
    
    def forward(self, x, feature):
        x = self.network(x).squeeze()
        x = self.linear(torch.cat([x, feature], 1))
        return x
    
netD = Discriminator()

In [7]:
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torchvision.utils as vutils

In [8]:
criterion = nn.MSELoss()

input = torch.FloatTensor(batchSize, 3, imageSize,imageSize)
noise = torch.FloatTensor(batchSize, nz, 1, 1)
feature = torch.FloatTensor(batchSize, 736)
fixed_noise = torch.FloatTensor(batchSize, nz, 1, 1).normal_(0, 1)
fixed_feature = dataset.feature[1]
fixed_feature = fixed_feature.repeat(batchSize, 1, 1, 1)

label_real = torch.FloatTensor(batchSize)
label_real_smooth = torch.FloatTensor(batchSize)
label_fake = torch.FloatTensor(batchSize)

netD.cuda()
netG.cuda()
criterion.cuda()
input, feature, noise = input.cuda(), feature.cuda(), noise.cuda()
label_real, label_real_smooth, label_fake = label_real.cuda(), label_real_smooth.cuda(), label_fake.cuda()
fixed_noise, fixed_feature = fixed_noise.cuda(), fixed_feature.cuda()

label_real.resize_(batchSize, 1).fill_(1)
label_fake.resize_(batchSize, 1).fill_(0)
label_real_smooth.resize_(batchSize, 1).fill_(0.8)
label_real = Variable(label_real)
label_fake = Variable(label_fake)
label_real_smooth = Variable(label_real_smooth)
print()

In [9]:
# netD.load_state_dict(torch.load(outf + 'netD_epoch_042.pth'))
# netG.load_state_dict(torch.load(outf + 'netG_epoch_042.pth'))

In [10]:
fixed_noise = Variable(fixed_noise)
fixed_feature = Variable(fixed_feature)

# setup optimizer
# optimizerD = optim.SGD(netD.parameters(), lr = lr, momentum=0.9)
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, beta2))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, beta2))
schedulerD = optim.lr_scheduler.MultiStepLR(optimizerD, milestones=[4, 7, 11, 17], gamma=0.87)
schedulerG = optim.lr_scheduler.MultiStepLR(optimizerG, milestones=[4, 7, 11, 17], gamma=0.87)
result_dict = {}
loss_D,loss_G,score_D,score_G1,score_G2 = [],[],[],[],[]

In [ ]:
import numpy as np
_d_ = None
def add_noise(x, d_fake):
    global _d_
    if _d_ is not None:
        _d_ = _d_ * 0.9 + torch.mean(d_fake).data[0] * 0.1
        strength = 0.2 * max(0, _d_ - 0.5)**2
        z = np.random.randn(*x.size()).astype(np.float32) * strength
        z = Variable(torch.from_numpy(z)).cuda()
        return x + z
    else:
        _d_ = 0.0
        return x


In [ ]:
niter = 100
d_fake_save = None
for epoch in range(niter):
    schedulerD.step()
    schedulerG.step()
    
    for i, data in enumerate(dataloader, 0):

        # train D
        netD.zero_grad()
        real_cpu, embed_cpu = data
        batch_size = real_cpu.size(0)

        real = real_cpu.cuda()
        embed = embed_cpu.cuda()
        input.resize_as_(real).copy_(real)
        feature.resize_as_(embed).copy_(embed)
        
        inputv = Variable(input)
        inputv = add_noise(inputv, d_fake_save)
        featurev = Variable(feature)
        
        d_real = netD(inputv, featurev)
        d_real_mean = d_real.data.mean()
        
        noise.resize_(batch_size, nz, 1, 1).normal_(0, 1)
        noisev = Variable(noise)
        fake = netG(noisev, featurev)
        
        d_fake_save = d_fake = netD(fake.detach(), featurev)
        d_fake_mean = d_fake.data.mean()
        
        loss_d = criterion(d_real, label_real_smooth) + criterion(d_fake, label_fake)
        loss_d.backward()
        optimizerD.step()
        
        # train G
        netG.zero_grad()
        d_fake = netD(fake, featurev)
        loss_g = criterion(d_fake, label_real.detach())
        loss_g.backward()
        optimizerG.step()

        if i%250 == 0:
            fake = netG(fixed_noise, fixed_feature)
            vutils.save_image(fake.data,
                    '%s/fake_samples_epoch_%03dstep_%04d.png' % (outf, epoch, i),
                    normalize=True)
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f'
                  % (epoch, niter, i, len(dataloader),
                     loss_d.data[0], loss_g.data[0], d_real_mean, d_fake_mean))
            
    torch.save(netG.state_dict(), '%s/netG_epoch_%03d.pth' % (outf, epoch))
    torch.save(netD.state_dict(), '%s/netD_epoch_%03d.pth' % (outf, epoch))

[0/100][0/3165] Loss_D: 0.6110 Loss_G: 0.8280 D(x): 0.0190 D(G(z)): 0.0200
[0/100][250/3165] Loss_D: 0.2822 Loss_G: 0.8090 D(x): 0.6102 D(G(z)): 0.4295
[0/100][500/3165] Loss_D: 0.3229 Loss_G: 0.4681 D(x): 0.4384 D(G(z)): 0.4300
[0/100][750/3165] Loss_D: 0.3036 Loss_G: 0.4802 D(x): 0.4418 D(G(z)): 0.4131
[0/100][1000/3165] Loss_D: 0.3301 Loss_G: 0.3799 D(x): 0.2997 D(G(z)): 0.2792
[0/100][1250/3165] Loss_D: 0.3034 Loss_G: 0.5132 D(x): 0.4593 D(G(z)): 0.4254
[0/100][1500/3165] Loss_D: 0.2484 Loss_G: 0.5659 D(x): 0.4699 D(G(z)): 0.3558
[0/100][1750/3165] Loss_D: 0.3120 Loss_G: 0.8984 D(x): 0.5705 D(G(z)): 0.4915
[0/100][2000/3165] Loss_D: 0.2594 Loss_G: 0.4935 D(x): 0.3647 D(G(z)): 0.2164
[0/100][2250/3165] Loss_D: 0.2692 Loss_G: 0.9173 D(x): 0.7184 D(G(z)): 0.4689
[0/100][2500/3165] Loss_D: 0.1719 Loss_G: 1.2312 D(x): 0.6878 D(G(z)): 0.3367
[0/100][2750/3165] Loss_D: 0.1851 Loss_G: 0.8431 D(x): 0.7092 D(G(z)): 0.3625
[0/100][3000/3165] Loss_D: 0.2312 Loss_G: 1.1346 D(x): 0.7828 D(G(z)):

[8/100][250/3165] Loss_D: 0.0410 Loss_G: 0.9800 D(x): 0.7707 D(G(z)): 0.0696
[8/100][500/3165] Loss_D: 0.0431 Loss_G: 0.9519 D(x): 0.8152 D(G(z)): 0.0799
[8/100][750/3165] Loss_D: 0.0478 Loss_G: 1.0518 D(x): 0.8077 D(G(z)): 0.1522
[8/100][1000/3165] Loss_D: 0.0266 Loss_G: 0.9762 D(x): 0.7505 D(G(z)): 0.0111
[8/100][1250/3165] Loss_D: 0.0300 Loss_G: 0.9256 D(x): 0.7098 D(G(z)): 0.0291
[8/100][1500/3165] Loss_D: 0.0437 Loss_G: 0.8894 D(x): 0.7054 D(G(z)): 0.0332
[8/100][1750/3165] Loss_D: 0.0059 Loss_G: 1.0190 D(x): 0.7998 D(G(z)): -0.0078
[8/100][2000/3165] Loss_D: 0.0784 Loss_G: 0.7515 D(x): 0.6089 D(G(z)): 0.0016
[8/100][2250/3165] Loss_D: 0.0347 Loss_G: 0.9590 D(x): 0.7816 D(G(z)): 0.1175
[8/100][2500/3165] Loss_D: 0.0264 Loss_G: 1.0179 D(x): 0.7840 D(G(z)): 0.0753
[8/100][2750/3165] Loss_D: 0.0207 Loss_G: 1.0473 D(x): 0.7900 D(G(z)): 0.0762
[8/100][3000/3165] Loss_D: 0.0546 Loss_G: 1.0081 D(x): 0.7388 D(G(z)): 0.1460
[9/100][0/3165] Loss_D: 0.0564 Loss_G: 1.0478 D(x): 0.7947 D(G(z))